# Data Preparation File for In-Car Recommendations
## Group Members: Jennifer Baker, Rachana Nagaraj Banakar, Devan Kreitzer, Patrick Maggio, Safrin Patil, Sravani Yadavalli

In this notebook, the dataset is initally loaded, encoded, and split into training and test sets so that data across the different models trained by group members remain consistent. We desire this consistency because we want to penultimately choose the best model by comparing the different model evaluations for predicting the 'Yes' for usage of a recommended coupon based on the survey results. 

### Import the necessary libaries

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
np.random.seed(42) 

### Reading In the File

In [2]:
df = pd.read_csv('in-vehicle-coupon-recommendation.csv')

### Cleaning Up Rows

In [3]:
df = df.dropna(axis=0, thresh=int(0.25*(df.shape[1]-1))) # axis=0 is row. thresh=int(0.25*(df.shape[1]-1)) means that if a row has less than 25% of the data, it will be dropped

### Cleaning Up Columns

Cleaning the columns by making sure that there are no spaces, and all capitalized for ease of reference

In [4]:
df.columns = [s.strip().replace(' ', '_').upper() for s in df.columns]

print(df.columns)

Index(['DESTINATION', 'PASSANGER', 'WEATHER', 'TEMPERATURE', 'TIME', 'COUPON',
       'EXPIRATION', 'GENDER', 'AGE', 'MARITALSTATUS', 'HAS_CHILDREN',
       'EDUCATION', 'OCCUPATION', 'INCOME', 'CAR', 'BAR', 'COFFEEHOUSE',
       'CARRYAWAY', 'RESTAURANTLESSTHAN20', 'RESTAURANT20TO50',
       'TOCOUPON_GEQ5MIN', 'TOCOUPON_GEQ15MIN', 'TOCOUPON_GEQ25MIN',
       'DIRECTION_SAME', 'DIRECTION_OPP', 'Y'],
      dtype='object')


CAR column was dropped because it was mostly NA, as shown in the below code cell. 

In [5]:
df.shape

(12684, 26)

Confirming no missing data from target 

In [6]:
df['Y'].isna().sum()

0

In [7]:
nulldict = {}

for col in df.columns:
    if(df[col].isna().sum() != 0):
        nulldict[col]=df[col].isna().sum()

print(nulldict)


{'CAR': 12576, 'BAR': 107, 'COFFEEHOUSE': 217, 'CARRYAWAY': 151, 'RESTAURANTLESSTHAN20': 130, 'RESTAURANT20TO50': 189}


Dropping Car column as most data is missing; not worth imputing

In [8]:
df = df.drop(columns='CAR')

The Encoding List below includes all the columns that we determined were categorical variables that needed encoding.

In [9]:
encodinglist = ["destination","passanger","weather","time","coupon","expiration","gender","age","maritalStatus","education", "occupation", "income", "Bar",'CoffeeHouse', 'CarryAway', 'RestaurantLessThan20', 'Restaurant20To50']
def all_upper(my_list):
    return [x.upper() for x in my_list]
encodinglist = all_upper(encodinglist)
print(encodinglist)



['DESTINATION', 'PASSANGER', 'WEATHER', 'TIME', 'COUPON', 'EXPIRATION', 'GENDER', 'AGE', 'MARITALSTATUS', 'EDUCATION', 'OCCUPATION', 'INCOME', 'BAR', 'COFFEEHOUSE', 'CARRYAWAY', 'RESTAURANTLESSTHAN20', 'RESTAURANT20TO50']


In [10]:
# Iterate through the columns in the list and perform one-hot encoding
for dummy in encodinglist:
    df = pd.get_dummies(
        df,
        prefix_sep="_",
        dummy_na=False,
        drop_first=False,
        columns=[dummy],
        dtype="int32",
    )

In [11]:
df.dtypes

TEMPERATURE               int64
HAS_CHILDREN              int64
TOCOUPON_GEQ5MIN          int64
TOCOUPON_GEQ15MIN         int64
TOCOUPON_GEQ25MIN         int64
                          ...  
RESTAURANT20TO50_1~3      int32
RESTAURANT20TO50_4~8      int32
RESTAURANT20TO50_gt8      int32
RESTAURANT20TO50_less1    int32
RESTAURANT20TO50_never    int32
Length: 110, dtype: object

Removes any special characters from column names

In [12]:
df.columns = [s.strip().replace(' ', '_').upper() for s in df.columns]
df.columns = [s.strip().replace('[', '_').upper() for s in df.columns]
df.columns = [s.strip().replace(']', '_').upper() for s in df.columns]
df.columns = [s.strip().replace(',', '_').upper() for s in df.columns]
df.columns = [s.strip().replace('<', '_').upper() for s in df.columns]

Creating the train-test split with the validation/test set as 0.2 because we want a large amount of data to train our models and 0.2

In [13]:
# construct datasets for analysis
target = 'Y'
predictors = list(df.columns)
predictors.remove(target)
X = df[predictors]
y = df[target]

train_X, test_X, train_y, test_y = train_test_split(X,y, test_size=0.2, random_state=1)


Imputing missing values

In [14]:
df.dtypes

TEMPERATURE               int64
HAS_CHILDREN              int64
TOCOUPON_GEQ5MIN          int64
TOCOUPON_GEQ15MIN         int64
TOCOUPON_GEQ25MIN         int64
                          ...  
RESTAURANT20TO50_1~3      int32
RESTAURANT20TO50_4~8      int32
RESTAURANT20TO50_GT8      int32
RESTAURANT20TO50_LESS1    int32
RESTAURANT20TO50_NEVER    int32
Length: 110, dtype: object

In [15]:
for column in train_X.select_dtypes(include=['object']):
    train_mode = train_X[column].value_counts().index[0]
    train_X[column] = train_X[column].fillna(0)
    test_X[column] = test_X[column].fillna(0)

# for the numeric columns, we choose the mean for our impute value.
imp = SimpleImputer(missing_values=np.nan, strategy='mean')

numeric_cols = ['TEMPERATURE', 'HAS_CHILDREN', 'TOCOUPON_GEQ5MIN','TOCOUPON_GEQ15MIN', 'TOCOUPON_GEQ25MIN','DIRECTION_SAME', 'DIRECTION_OPP']
imp.fit(train_X[numeric_cols])
train_X[numeric_cols] = imp.transform(train_X[numeric_cols])
test_X[numeric_cols] = imp.transform(test_X[numeric_cols])

Exporting Training/Test Splits to Be Shared By Team

In [16]:
train_X.to_csv('train_X_In-Car-Rec.csv', index = False)
train_y.to_csv('train_y_In-Car-Rec.csv', index = False)
test_X.to_csv('test_X_In-Car-Rec.csv', index = False)
test_y.to_csv('test_y_In-Car-Rec.csv', index = False)